### Задание 1
План:
- найдем средние рейтинги
- сопоставим фильм и рейтинг
- присвоим 'class' каждому фильму в зависимости от его рейтинга

In [1]:
import pandas as pd
from pathlib import Path


BASE_DIR = Path('/home/jovyan/work/homework') 
MOVIES_PATH = BASE_DIR / 'data/ml-latest-small/movies.csv'
RATINGS_PATH = BASE_DIR / 'data/ml-latest-small/ratings.csv'

In [2]:
ratings = pd.read_csv(RATINGS_PATH)
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [3]:
mean_ratings = ratings.groupby(by='movieId')['rating'].mean()
mean_ratings

movieId
1         3.872470
2         3.401869
3         3.161017
4         2.384615
5         3.267857
            ...   
161944    5.000000
162376    4.500000
162542    5.000000
162672    3.000000
163949    5.000000
Name: rating, Length: 9066, dtype: float64

In [4]:
MOVIES_PATH = BASE_DIR / 'data/ml-latest-small/movies.csv'
movies = pd.read_csv(MOVIES_PATH)
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
movies_and_rating = pd.merge(movies, mean_ratings, on='movieId')
movies_and_rating.head()

,movieId,title,genres,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.401869
2,3,Grumpier Old Men (1995),Comedy|Romance,3.161017
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615
4,5,Father of the Bride Part II (1995),Comedy,3.267857


In [6]:
# оценка 2 и ниже — низкий рейтинг
# оценка 4 и ниже — средний рейтинг
# оценка 4.5 и 5 — высокий рейтинг

def classify(rating):
    if rating <= 2:
        return 'low'

    if rating <= 4:
        return 'medium'

    return 'high'

movies_and_rating['class'] = movies_and_rating['rating'].apply(classify)
movies_and_rating.head()

,movieId,title,genres,rating,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470,medium
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.401869,medium
2,3,Grumpier Old Men (1995),Comedy|Romance,3.161017,medium
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615,medium
4,5,Father of the Bride Part II (1995),Comedy,3.267857,medium


In [7]:
HIGH_CLASS = movies_and_rating['class'] == 'high'
movies_and_rating[HIGH_CLASS].head()

,movieId,title,genres,rating,class
25,26,Othello (1995),Drama,4.100000,high
27,28,Persuasion (1995),Drama|Romance,4.083333,high
28,29,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi,4.025000,high
29,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Crime|Drama,4.050000,high
39,41,Richard III (1995),Drama|War,4.021739,high


In [8]:
MEDIUM_CLASS = movies_and_rating['class'] == 'medium'
movies_and_rating[MEDIUM_CLASS].head()

,movieId,title,genres,rating,class
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.872470,medium
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.401869,medium
2,3,Grumpier Old Men (1995),Comedy|Romance,3.161017,medium
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2.384615,medium
4,5,Father of the Bride Part II (1995),Comedy,3.267857,medium


In [9]:
LOW_CLASS = movies_and_rating['class'] == 'low'
movies_and_rating[LOW_CLASS].head()

,movieId,title,genres,rating,class
35,37,Across the Sea of Time (1995),Documentary|IMAX,2.000000,low
36,38,It Takes Two (1995),Children|Comedy,2.000000,low
62,66,Lawnmower Man 2: Beyond Cyberspace (1996),Action|Sci-Fi|Thriller,2.000000,low
81,87,Dunston Checks In (1996),Children|Comedy,1.666667,low
88,96,In the Bleak Midwinter (1995),Comedy|Drama,1.000000,low


### Задание 2

In [10]:
import re

KEYWORDS_PATH = BASE_DIR / 'data/ml-latest-small/keywords.csv'
keywords = pd.read_csv(KEYWORDS_PATH)
keywords.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [11]:
geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

geo_data_classificator = {}
for region, cities in geo_data.items():
    for city in cities:
        geo_data_classificator[city] = region

geo_data_classificator

{'москва': 'Центр',
 'тула': 'Центр',
 'ярославль': 'Центр',
 'петербург': 'Северо-Запад',
 'псков': 'Северо-Запад',
 'мурманск': 'Северо-Запад',
 'владивосток': 'Дальний Восток',
 'сахалин': 'Дальний Восток',
 'хабаровск': 'Дальний Восток'}

In [12]:
def define_region(keyword):
    words = re.findall(r'\w+', keyword)
    
    for word in words:
        if word in geo_data_classificator:
            return geo_data_classificator[word]

    return 'undefined'
        
keywords['region'] = keywords['keyword'].apply(define_region)
keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


In [13]:
keywords_with_region = keywords['region'] != 'undefined'
keywords[keywords_with_region].head()

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1236,банк санкт петербург,174375,Северо-Запад


In [14]:
keywords[~keywords_with_region].head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


### Оптимизация

In [15]:
def define_region_iterate_over_words(keyword):
    words = re.findall(r'\w+', keyword)
    
    for word in words:
        if word in geo_data_classificator:
            return geo_data_classificator[word]

    return 'undefined'

def define_region_iterate_over_cities(keyword):
    for city, region in geo_data_classificator.items():
        if city in keyword:
            return region

    return 'undefined'

In [16]:
%%timeit

keywords['region'] = keywords['keyword'].apply(define_region_iterate_over_words)

96.4 ms ± 3.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
%%timeit

keywords['region'] = keywords['keyword'].apply(define_region_iterate_over_cities)

67.6 ms ± 3.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Хотя итерирование по городам выигрывает по времени в данном случае, это может измениться, если количество городов увеличится, потому что его сложность - O(n), где n - количество городов, поэтому время работы прямо пропорционально зависит от количества городов.

In [18]:
geo_data_classificator = {
    'москва': 'Центр',
    'тула': 'Центр',
    'ярославль': 'Центр',
    'брянск': 'Центр',
    'калуга': 'Центр',
    'орёл': 'Центр',
    
    'петербург': 'Северо-Запад',
    'псков': 'Северо-Запад',
    'мурманск': 'Северо-Запад',
    'вологда': 'Северо-Запад',
    'архангельск': 'Северо-Запад',
    'карелия': 'Северо-Запад',
    
    'владивосток': 'Дальний Восток',
    'сахалин': 'Дальний Восток',
    'хабаровск': 'Дальний Восток',
    'камчатка': 'Дальний Восток',
    'якутск': 'Дальний Восток',
    'благовещенск': 'Дальний Восток'
}

In [19]:
%%timeit

keywords['region'] = keywords['keyword'].apply(define_region_iterate_over_words)

109 ms ± 5.28 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%%timeit

keywords['region'] = keywords['keyword'].apply(define_region_iterate_over_cities)

132 ms ± 1.61 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


С увеличением количества городов в 2 раза:
- итерация по городам -  время на обработку выросло в ~2 раза, что соответвует ожиданию
- итерация по словам - время на обработку немного увеличилось (погрешность, внешние факторы)